# Import Libraries

In [1]:
pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install contractions


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
import emoji
import pandas as pd 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import seaborn as sns
import contractions
import warnings
warnings.filterwarnings("ignore")

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop

In [6]:
# Download NLTK resources if not already done
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Read the dataset 

In [7]:
# Read the CSV file with specified column names
df = pd.read_csv("/kaggle/input/positive-vs-negative-tweets/clean_data.csv")

In [8]:
df.head (10)

,text,target,tweet_length
0,"['bummer', 'shoulda', 'got', 'david', 'carr', ...",0,39
1,"['upset', 'cannot', 'update', 'facebook', 'tex...",0,76
2,"['dived', 'many', 'time', 'ball', 'managed', '...",0,47
3,"['whole', 'body', 'feel', 'itchy', 'like', 'fi...",0,31
4,"['behaving', 'mad', 'cannot', 'see']",0,23
5,"['whole', 'crew']",0,10
6,"['need', 'hug']",0,8
7,"['hey', 'long', 'time', 'see', 'yes', 'rain', ...",0,50
8,['nope'],0,4
9,"['que', 'muera']",0,9


In [9]:
# Select 20,000 instances of 0
df_1 = df[df['target'] == 1].sample(n=10000, random_state=42)

# Select 20,000 instances of 0
df_0 = df[df['target'] == 0].sample(n=10000, random_state=42)

# Concatenate the two DataFrames
new_df = pd.concat([df_1, df_0], ignore_index=True)

# Shuffle the new DataFrame
new_df = new_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [10]:
import gc

# Assuming df is the name of your DataFrame
del df

# Force garbage collection
gc.collect()


13

In [11]:
# Assuming df is the name of your DataFrame
del df_0

# Force garbage collection
gc.collect()

0

In [12]:
# Assuming df is the name of your DataFrame
del df_1

# Force garbage collection
gc.collect()

0

In [13]:
new_df.head(10)

,text,target,tweet_length
0,"['meeko', 'get', 'surgery', 'poor', 'kitty']",0,28
1,"['really', 'excited', 'see', 'brother', 'si', ...",1,42
2,"['nj', 'transit', 'sympathy']",1,19
3,"['spend', 'sunday', 'night', 'love', 'rove']",1,28
4,"['hoping', 'wed', 'follower']",0,19
5,"['cannot', 'even', 'explain', 'much', 'miss']",0,29
6,"['ood', 'snapper', 'tarakihi', 'kingfish', 'po...",0,42
7,"['six', 'hour', 'work', 'vacation', 'time', 'h...",1,45
8,"['finished', 'watching', 'footy', 'origin', 'a...",0,71
9,"['call', 'text', 'next', 'hour', 'phone', 'upd...",1,38


#  BERT

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split


In [15]:
texts = new_df['text'].tolist()
labels = new_df['target'].tolist() 
    

In [16]:
# Dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            return_tensors='pt',
            max_length=self.max_length,
            padding='max_length',
            truncation=True
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)  # Ensure labels are long type
        }


In [17]:
# BERT Classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits


In [18]:
# Training function
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        
        loss.backward()
        optimizer.step()
        scheduler.step()


In [19]:
# Evaluation function
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)


In [20]:
# Predict function for sentiment
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(
        text, 
        return_tensors='pt', 
        max_length=max_length, 
        padding='max_length', 
        truncation=True
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    
    return "positive" if preds.item() == 1 else "negative"


In [21]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 128
batch_size = 16 
num_epochs = 4
learning_rate = 2e-5


In [22]:
# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [23]:
# Tokenizer
tokenizer = BertTokenizer.from_pretrained(bert_model_name)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [24]:
# Datasets and Dataloaders
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)


In [25]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [26]:
# Model, optimizer, and scheduler setup
model = BERTClassifier(bert_model_name, num_classes).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

# Model

In [27]:
import time

# Training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start time of the epoch
    print(f"Epoch {epoch + 1}/{num_epochs}")

    train(model, train_dataloader, optimizer, scheduler, device)
    
    accuracy, report = evaluate(model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(report)

    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Calculate duration
    print(f"Epoch {epoch + 1} Duration: {epoch_duration:.2f} seconds")


Epoch 1/4
Validation Accuracy: 0.7548
              precision    recall  f1-score   support

           0       0.73      0.79      0.76      1945
           1       0.78      0.72      0.75      2055

    accuracy                           0.75      4000
   macro avg       0.76      0.76      0.75      4000
weighted avg       0.76      0.75      0.75      4000

Epoch 1 Duration: 214.12 seconds
Epoch 2/4
Validation Accuracy: 0.7620
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      1945
           1       0.78      0.75      0.76      2055

    accuracy                           0.76      4000
   macro avg       0.76      0.76      0.76      4000
weighted avg       0.76      0.76      0.76      4000

Epoch 2 Duration: 213.79 seconds
Epoch 3/4
Validation Accuracy: 0.7502
              precision    recall  f1-score   support

           0       0.74      0.75      0.75      1945
           1       0.76      0.75      0.76      2055

  

# save the model 

In [28]:
import torch

# Saving the model and tokenizer
def save_model(model, tokenizer, model_path, tokenizer_path):
    # Save model weights
    torch.save(model.state_dict(), model_path)
    
    # Save tokenizer
    tokenizer.save_pretrained(tokenizer_path)

# Paths where you want to save the model and tokenizer
model_path = "bert_classifier_model.pth"
tokenizer_path = "bert_tokenizer/"

# Save the model and tokenizer
save_model(model, tokenizer, model_path, tokenizer_path)


# Loading the Model and Tokenizer 

In [29]:
import torch
from transformers import BertTokenizer

# Loading the model and tokenizer
def load_model(bert_model_name, model_path, tokenizer_path, num_classes, device):
    # Load the tokenizer
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    
    # Initialize model
    model = BERTClassifier(bert_model_name, num_classes).to(device)
    
    # Load model weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    
    model.eval()  # Set the model to evaluation mode
    return model, tokenizer

# Paths to the saved model and tokenizer
model_path = "bert_classifier_model.pth"
tokenizer_path = "bert_tokenizer/"

# Load the model and tokenizer
loaded_model, loaded_tokenizer = load_model(bert_model_name, model_path, tokenizer_path, num_classes, device)


# Testing

In [30]:
# Example text to predict sentiment
example_text = "I love using BERT for natural language processing tasks."

# Use the loaded model and tokenizer to predict sentiment
def predict_sentiment_from_loaded_model(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(
        text, 
        return_tensors='pt', 
        max_length=max_length, 
        padding='max_length', 
        truncation=True
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    
    return "positive" if preds.item() == 1 else "negative"

# Predict sentiment using the loaded model
sentiment = predict_sentiment_from_loaded_model(example_text, loaded_model, loaded_tokenizer, device)
print(f"Sentiment of the example text: {sentiment}")


Sentiment of the example text: positive


#  gradio


In [31]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 66.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 81.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.9.0 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have goo

In [32]:
import gradio as gr
import torch
from transformers import BertTokenizer
import torch.nn as nn

# Define the model architecture (make sure this matches your saved model)
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

# Function to load the model and tokenizer
def load_model(bert_model_name, model_path, tokenizer_path, num_classes, device):
    # Load the tokenizer
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    
    # Initialize model
    model = BERTClassifier(bert_model_name, num_classes).to(device)
    
    # Load model weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()  # Set the model to evaluation mode
    return model, tokenizer

# Paths to the saved model and tokenizer
model_path = "bert_classifier_model.pth"
tokenizer_path = "bert_tokenizer/"
bert_model_name = 'bert-base-uncased'
num_classes = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model, tokenizer = load_model(bert_model_name, model_path, tokenizer_path, num_classes, device)

# Function to predict sentiment
def predict_sentiment(text):
    encoding = tokenizer(
        text,
        return_tensors='pt',
        max_length=128,
        padding='max_length',
        truncation=True
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    
    return "Positive" if preds.item() == 1 else "Negative"

# Set up the Gradio interface
interface = gr.Interface(
    fn=predict_sentiment, 
    inputs="text", 
    outputs="text", 
    title="BERT Sentiment Classifier",
    description="Enter a sentence to predict its sentiment (positive/negative)."
)

# Launch the interface
interface.launch(share=True)  # You can also share the link publicly


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://844084929f9edcd918.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
